### Test methods that work with accounts

In [1]:
from dexter.DB import DB, Account, Entry, Transaction, Document, Tag

import re

In [2]:
DB.init()

In [3]:
DB.open('dev')

### Match Account Names

In [4]:
Account.objects(name__contains='checking')

[<Account: checking assets>]

In [5]:
Account.objects(name__contains='food')

[<Account: expenses:food expenses>, <Account: groceries expenses>, <Account: restaurant expenses>]

In [6]:
for a in DB.find_account('food'):
    print(a)

expenses:food expenses
groceries expenses
restaurant expenses


In [7]:
DB.account_name_parts()

{'assets',
 'bank',
 'car',
 'chase',
 'checking',
 'entertainment',
 'equity',
 'expenses',
 'food',
 'fuel',
 'groceries',
 'home',
 'household',
 'income',
 'interest',
 'liabilities',
 'mortgage',
 'payment',
 'restaurant',
 'savings',
 'utility',
 'visa',
 'yard',
 'yoyodyne'}

In [8]:
DB.account_name_parts('expenses')

set()

In [9]:
dct = DB.account_names()

In [10]:
dct

{'equity': {'equity'},
 'assets:bank:checking': {'assets:bank:checking'},
 'checking': {'assets:bank:checking'},
 'assets': {'assets:bank:checking', 'assets:bank:savings'},
 'bank': {'assets:bank:checking', 'assets:bank:savings'},
 'assets:bank:savings': {'assets:bank:savings'},
 'savings': {'assets:bank:savings'},
 'expenses:car': {'expenses:car'},
 'expenses': {'expenses:car',
  'expenses:car:fuel',
  'expenses:car:payment',
  'expenses:entertainment',
  'expenses:food',
  'expenses:food:groceries',
  'expenses:food:restaurant',
  'expenses:home',
  'expenses:home:household',
  'expenses:home:mortgage',
  'expenses:home:utility',
  'expenses:home:yard'},
 'car': {'expenses:car', 'expenses:car:fuel', 'expenses:car:payment'},
 'expenses:car:payment': {'expenses:car:payment'},
 'payment': {'expenses:car:payment'},
 'expenses:car:fuel': {'expenses:car:fuel'},
 'fuel': {'expenses:car:fuel'},
 'expenses:entertainment': {'expenses:entertainment'},
 'entertainment': {'expenses:entertainment'

In [11]:
dct['expenses']

{'expenses:car',
 'expenses:car:fuel',
 'expenses:car:payment',
 'expenses:entertainment',
 'expenses:food',
 'expenses:food:groceries',
 'expenses:food:restaurant',
 'expenses:home',
 'expenses:home:household',
 'expenses:home:mortgage',
 'expenses:home:utility',
 'expenses:home:yard'}

In [12]:
len(dct['expenses'])

12

In [13]:
dct = DB.account_names('expenses')

In [14]:
dct

{}

In [15]:
len(dct)

0

In [17]:
DB.account_groups()

['equity',
 'assets:bank:checking',
 'assets:bank:savings',
 'expenses:car',
 'expenses:car:payment',
 'expenses:car:fuel',
 'expenses:entertainment',
 'expenses:food',
 'expenses:food:groceries',
 'expenses:food:restaurant',
 'expenses:home',
 'expenses:home:mortgage',
 'expenses:home:household',
 'expenses:home:utility',
 'expenses:home:yard',
 'income:yoyodyne',
 'income:interest',
 'liabilities:chase:visa']

In [18]:
DB.account_groups(['expenses'])

['expenses:car',
 'expenses:car:payment',
 'expenses:car:fuel',
 'expenses:entertainment',
 'expenses:food',
 'expenses:food:groceries',
 'expenses:food:restaurant',
 'expenses:home',
 'expenses:home:mortgage',
 'expenses:home:household',
 'expenses:home:utility',
 'expenses:home:yard']

In [19]:
DB.account_groups(['expenses:1'])

['expenses.*']

In [20]:
DB.account_groups(['expenses:2'])

['expenses:car.*',
 'expenses:entertainment.*',
 'expenses:food.*',
 'expenses:home.*']

In [21]:
DB.account_groups(['expenses:3'])

['expenses:car',
 'expenses:car:payment.*',
 'expenses:car:fuel.*',
 'expenses:entertainment',
 'expenses:food',
 'expenses:food:groceries.*',
 'expenses:food:restaurant.*',
 'expenses:home',
 'expenses:home:mortgage.*',
 'expenses:home:household.*',
 'expenses:home:utility.*',
 'expenses:home:yard.*']

In [22]:
DB.account_groups(['expenses:food'])

['expenses:food', 'expenses:food:groceries', 'expenses:food:restaurant']

In [23]:
DB.account_groups(['expenses:food:2'])

['expenses:food.*']

In [24]:
DB.account_groups(['expenses:food:3'])

['expenses:food', 'expenses:food:groceries.*', 'expenses:food:restaurant.*']

In [25]:
DB.account_groups(['expenses','income'])

['expenses:car',
 'expenses:car:payment',
 'expenses:car:fuel',
 'expenses:entertainment',
 'expenses:food',
 'expenses:food:groceries',
 'expenses:food:restaurant',
 'expenses:home',
 'expenses:home:mortgage',
 'expenses:home:household',
 'expenses:home:utility',
 'expenses:home:yard',
 'income:yoyodyne',
 'income:interest']

In [26]:
DB.account_groups(['expenses:1','income:1'])

['expenses.*', 'income.*']

In [27]:
DB.account_groups(['expenses:2','income:2'])

['expenses:car.*',
 'expenses:entertainment.*',
 'expenses:food.*',
 'expenses:home.*',
 'income:yoyodyne.*',
 'income:interest.*']

In [28]:
Account.objects(name__exact='expenses:food:groceries')

[<Account: groceries expenses>]

In [29]:
Account.objects(abbrev__exact='groceries')

[<Account: groceries expenses>]

In [30]:
s = 'expenses:food:groceries'

In [31]:
if acct := Account.objects(name__exact=s) or Account.objects(abbrev__exact=s):
    print(acct[0].name)

expenses:food:groceries


In [32]:
DB.fullname('expenses:food:groceries')

'expenses:food:groceries'

In [33]:
DB.fullname('groceries')

'expenses:food:groceries'

In [34]:
DB.fullname('groc')

In [35]:
Account.objects.get(name='assets:bank:checking')

<Account: checking assets>

In [36]:
Account.objects.get(abbrev='checking')

<Account: checking assets>

Bummer -- would be nice if this returned None instead of raising an exception:

In [37]:
# Account.objects.get(abbrev='foo')

In [38]:
Account.DoesNotExist

dexter.DB.DoesNotExist

In [39]:
Account.objects.first()

<Account: equity equity>

In [40]:
def abbrev(a):
    try:
        acct = Account.objects.get(name=a)
        res = acct.abbrev
    except Account.DoesNotExist:
        res = a
    return res

In [41]:
abbrev('assets:bank:checking')

'checking'

In [42]:
abbrev('assets:bank:whatever')

'assets:bank:whatever'

In [43]:
abbrev('expenses:personal:bob')

'expenses:personal:bob'

### Balances

In [44]:
DB.select(Entry, account='expenses:food*')

[<Entry: <En 2024-04-26 expenses:food:groceries +$15.0 []>>, <Entry: <En 2024-04-24 expenses:food:groceries +$15.0 []>>, <Entry: <En 2024-04-22 expenses:food:restaurant +$50.0 []>>, <Entry: <En 2024-04-23 expenses:food:groceries +$15.0 []>>, <Entry: <En 2024-04-01 expenses:food:restaurant +$70.0 []>>, <Entry: <En 2024-04-03 expenses:food:restaurant -$35.0 []>>, <Entry: <En 2024-04-12 expenses:food:groceries +$65.0 []>>]

In [45]:
for e in DB.select(Entry, account='expenses:food*'):
    print(e)

<En 2024-04-26 expenses:food:groceries +$15.0 []>
<En 2024-04-24 expenses:food:groceries +$15.0 []>
<En 2024-04-22 expenses:food:restaurant +$50.0 []>
<En 2024-04-23 expenses:food:groceries +$15.0 []>
<En 2024-04-01 expenses:food:restaurant +$70.0 []>
<En 2024-04-03 expenses:food:restaurant -$35.0 []>
<En 2024-04-12 expenses:food:groceries +$65.0 []>


In [46]:
for t in DB.select(Transaction, tag='#budget'):
    print(t)

In [47]:
DB.balance('food')

195.0

In [48]:
DB.balance('food', budgets=False)

195.0

In [49]:
DB.balance('food', ending='2024-01-31')

0

In [50]:
DB.balance('food', ending='2024-01-31', budgets=False)

0

In [51]:
for e in DB.select(Entry, account='expenses:car'):
    print(e)

<En 2024-04-05 expenses:car:payment +$400.0 []>
<En 2024-04-02 expenses:car +$5.0 []>
<En 2024-04-03 expenses:car:fuel +$50.0 []>


In [53]:
DB.select(Entry, account='expenses:car').sum('amount')

455.0

In [54]:
type(DB.select(Entry, account='expenses:car'))

mongoengine.queryset.queryset.QuerySet